In [1]:
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder



In [2]:
# Đọc file CSV
df = pd.read_csv('/kaggle/input/ai-1904-dpl-302-m-butterfly-image-classification/Training_set.csv')

# Giả sử CSV có cột 'filename' và 'label'
image_folder = '/kaggle/input/ai-1904-dpl-302-m-butterfly-image-classification/train/train'  # Thư mục chứa ảnh, không phải file CSV

# Encode label nếu chưa
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# Split train/val
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)



# Dataset class
class ButterflyDataset(Dataset):
    def __init__(self, df, image_folder, transform=None):
        self.df = df
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_folder, row['filename'])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(row['label_encoded'], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



In [4]:

image_folder = '/kaggle/input/ai-1904-dpl-302-m-butterfly-image-classification/train/train'  # ví dụ './train_images/'

# Dataloader
train_dataset = ButterflyDataset(train_df, image_folder, transform)
val_dataset = ButterflyDataset(val_df, image_folder, transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [5]:
# Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)
num_classes = df['label_encoded'].nunique()
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss + Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train thử #10 
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    # Test trên val
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"Val Accuracy: {acc*100:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 207MB/s]


Epoch 1/15, Train Loss: 1.8539
Val Accuracy: 87.70%
Epoch 2/15, Train Loss: 0.4043
Val Accuracy: 90.40%
Epoch 3/15, Train Loss: 0.1665
Val Accuracy: 90.70%
Epoch 4/15, Train Loss: 0.0899
Val Accuracy: 93.20%
Epoch 5/15, Train Loss: 0.0679
Val Accuracy: 91.70%
Epoch 6/15, Train Loss: 0.0827
Val Accuracy: 90.10%
Epoch 7/15, Train Loss: 0.0896
Val Accuracy: 91.40%
Epoch 8/15, Train Loss: 0.0758
Val Accuracy: 90.70%
Epoch 9/15, Train Loss: 0.0624
Val Accuracy: 89.30%
Epoch 10/15, Train Loss: 0.0736
Val Accuracy: 91.10%
Epoch 11/15, Train Loss: 0.0432
Val Accuracy: 91.40%
Epoch 12/15, Train Loss: 0.0546
Val Accuracy: 92.50%
Epoch 13/15, Train Loss: 0.0636
Val Accuracy: 89.10%
Epoch 14/15, Train Loss: 0.0348
Val Accuracy: 93.80%
Epoch 15/15, Train Loss: 0.0185
Val Accuracy: 93.20%


In [6]:
# Save thử model
torch.save(model.state_dict(), 'resnet50_butterfly_trial_2.pth')

In [7]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

In [8]:
# 📌 1. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# 🖼️ 2. Dataset test
class TestDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_paths = sorted([os.path.join(image_folder, fname) for fname in os.listdir(image_folder)])
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        filename = os.path.basename(img_path)
        return image, filename


In [10]:
# 🎨 3. Transform giống lúc train
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [11]:
# 📂 4. Load test dataset
test_dir = "/kaggle/input/ai-1904-dpl-302-m-butterfly-image-classification/test/test"  # sửa nếu folder khác
test_dataset = TestDataset(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [12]:
# 🧠 5. Load model
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 75)  # 75 lớp
model.load_state_dict(torch.load("resnet50_butterfly_trial_2.pth", map_location=device))
model = model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# 6️⃣ Load label encoder
train_df = pd.read_csv('/kaggle/input/ai-1904-dpl-302-m-butterfly-image-classification/Training_set.csv')
le = LabelEncoder()
le.fit(train_df['label'])

LabelEncoder()

In [14]:
# 7️⃣ Predict và decode
predictions = []
with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        labels = le.inverse_transform(preds.cpu().numpy())
        for fname, label in zip(filenames, labels):
            predictions.append((fname, label))


In [15]:
# 💾 8. Lưu kết quả ra CSV
submission = pd.DataFrame(predictions, columns=["filename", "label"])
submission.to_csv("submission_2.csv", index=False)
print("✅ Dự đoán hoàn tất. Kết quả lưu trong 'submission_2.csv'")

✅ Dự đoán hoàn tất. Kết quả lưu trong 'submission_2.csv'
